In [24]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
import duckdb

In [2]:
engine = create_engine('sqlite:////Users/paz/Documents/github/home-property-tax/wickersham/tax_year_2023/database/home_property_tax.db')

# The Market

In [109]:
market_trends_query = '''
    select
		yr as year,
		max(case when acct = '931920000058' then tot_mkt_val end) as subject_mkt_value,
        avg(case when lgl_2 = 'BRIARGROVE PARK SEC 1' then tot_mkt_val end) as legal_description_mkt_value,
		avg(case when neighborhood_code = 7947 then tot_mkt_val end) as neighborhood_mkt_value,
        avg(case when neighborhood_grp = 1117 then tot_mkt_val end) as neighborhood_group_mkt_value,
        avg(case when map_facet = '4956D' then tot_mkt_val end) as map_facet_mkt_value,
		avg(tot_mkt_val) as total_market_value
	from real_acct ra 
	where TRUE 
		and state_class = 'A1'
	group BY 
		1
	order BY 
		1 asc
    '''

In [110]:
market_trends_df = pd.read_sql_query(market_trends_query, engine)

In [111]:
fig = px.line(market_trends_df, x='year', y=['subject_mkt_value','legal_description_mkt_value','neighborhood_mkt_value','map_facet_mkt_value', 'neighborhood_group_mkt_value', 'total_market_value'], template='simple_white')
fig.show()

# The Neighborhood

In [176]:
neighborhood_query = '''
    select
        acct as account,
        yr as year,
        neighborhood_code,
        lgl_2 as neighborhood,
        neighborhood_grp as neighborhood_group,
        map_facet,
        site_addr_1 as street_address,
        site_addr_3 as zipcode,
        tot_mkt_val as market_value,
        prior_tot_mkt_val as prior_market_value,
        tot_mkt_val - prior_tot_mkt_val as change_in_market_value,
        tot_mkt_val / prior_tot_mkt_val - 1 as pct_change_in_market_value,
    from real_acct
    where
        true
        and state_class = 'A1'
        and coalesce(new_construction_val,0) = 0
        and yr = '2023'
        and neighborhood_grp = 1117
'''

# Remodeling

In [144]:
remodeling_query = '''
    with cte_2022 as (
        select
            acct as account,
            yr as year,
            neighborhood_code,
            lgl_2 as neighborhood,
            neighborhood_grp as neighborhood_group,
            map_facet,
            site_addr_1 as street_address,
            site_addr_3 as zipcode,
            tot_mkt_val as market_value,
            prior_tot_mkt_val as prior_market_value,
            new_construction_val as new_construction_value,
            tot_mkt_val - prior_tot_mkt_val as change_in_market_value,
            tot_mkt_val / prior_tot_mkt_val - 1 as pct_change_in_market_value,
            new_construction_val / prior_tot_mkt_val as new_construction_pct_total_prior_market_value,
            new_construction_val / (tot_mkt_val - prior_tot_mkt_val) as new_construction_pct_change_in_market_value
        from real_acct
        where
            true
            and state_class = 'A1'
            and new_construction_val > 20000
            and prior_market_value between 250000 and 700000
            and yr = '2022'
    ),
    cte_2023 as (
        SELECT 
            acct,
            tot_mkt_val
        from
            real_acct ra 
            inner join cte_2022 on cte_2022.account = ra.acct
        WHERE
            yr = '2023'
    )
    SELECT 
        cte_2022.*,
        cte_2023.tot_mkt_val as next_market_value,
        cte_2023.tot_mkt_val / cte_2022.market_value - 1 as next_market_value_pct_change
    from cte_2022
        left join cte_2023 on cte_2023.acct = cte_2022.account
'''

In [145]:
remodeling_df = pd.read_sql_query(remodeling_query, engine)

In [146]:
# remodeling_df_filtered = remodeling_df.loc[remodeling_df['neighborhood_code'].isin([7946, 7947, 7946.03])]
remodeling_df_filtered = remodeling_df.loc[remodeling_df['neighborhood_group'].isin([1117])]

In [147]:
fig = px.histogram(remodeling_df_filtered, x="next_market_value_pct_change", marginal="box", hover_data=remodeling_df.columns)
fig.show()

In [148]:
remodeling_df_filtered.sort_values(by=['pct_change_in_market_value'], ascending=False)

,account,year,neighborhood_code,neighborhood,neighborhood_group,map_facet,street_address,zipcode,market_value,prior_market_value,new_construction_value,change_in_market_value,pct_change_in_market_value,new_construction_pct_total_prior_market_value,new_construction_pct_change_in_market_value,next_market_value,next_market_value_pct_change
1883,932180000766,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10030 GREEN TREE RD,77042,946687.0,368496.0,486067.0,578191.0,1.569056,1.319056,0.840669,1315375.0,0.389451
1884,932180000769,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10110 GREEN TREE RD,77042,1345000.0,528234.0,779035.0,816766.0,1.546220,1.474791,0.953804,1674576.0,0.245038
1877,931980000252,2022,7947.00,BRIARGROVE PARK SEC 1,1117,4956B,1903 BRIARPARK DR,77042,950000.0,380000.0,559062.0,570000.0,1.500000,1.471216,0.980811,NaN,NaN
1887,932200000821,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10019 BRIAR DR,77042,841646.0,437427.0,277189.0,404219.0,0.924083,0.633681,0.685740,868062.0,0.031386
1890,932270000018,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10227 PINE FOREST RD,77042,1009103.0,597757.0,560034.0,411346.0,0.688149,0.936892,1.361467,1069834.0,0.060183
1879,932100000533,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4956B,10018 LONGMONT DR,77042,663879.0,410805.0,253074.0,253074.0,0.616044,0.616044,1.000000,1073233.0,0.616609
1872,931920000058,2022,7947.00,BRIARGROVE PARK SEC 1,1117,4956D,10027 WICKERSHAM LN,77042,652735.0,414458.0,174829.0,238277.0,0.574912,0.421826,0.733722,715818.0,0.096644
1891,932280000040,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957C,10331 CANDLEWOOD DR,77042,796200.0,520000.0,85466.0,276200.0,0.531154,0.164358,0.309435,818000.0,0.027380
1880,932100000549,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10015 VALLEY FORGE DR,77042,604893.0,410153.0,180883.0,194740.0,0.474798,0.441013,0.928844,1084691.0,0.793195
1886,932190000793,2022,7946.00,BRIARGROVE PARK SEC 1,1117,4957D,10118 SHADY RIVER DR,77042,988000.0,681110.0,344843.0,306890.0,0.450573,0.506296,1.123670,1179805.0,0.194135


# Comps

In [169]:
hcad_equity_comps = ['931980000216','931930000073','0931980000232','931980000235','931980000238']
hcad_sale_comps = ['931980000223','931990000274','931980000216','931940000110','932020000340']
zwicker_comps = ['931930000077','931940000096','931940000110','931980000240','932000000311']
subject = ['931920000058']
full_comp_list = hcad_equity_comps + hcad_sale_comps + zwicker_comps + subject
full_comp_tuple = tuple(full_comp_list)
hcad_equity_tuple = tuple(hcad_equity_comps)
hcad_sale_tuple = tuple(hcad_sale_comps)
zwicker_tuple = tuple(zwicker_comps)
subject_string = subject[0]

In [172]:
comp_query = f'''
    with cte_2022 as (
        select
            acct as account,
            case when acct in {hcad_equity_tuple} then true else false end as is_hcad_equity_comp,
            case when acct in {hcad_sale_tuple} then true else false end as is_hcad_sale_comp,
            case when acct in {zwicker_tuple} then true else false end as is_zwicker_comp,
            case when acct = '{subject_string}' then true else false end as is_subject,
            site_addr_1 as street_address,
            tot_mkt_val as market_value_2022,
            prior_tot_mkt_val as market_value_2021,
            new_construction_val as new_construction_value_2022,
            tot_mkt_val - prior_tot_mkt_val as change_in_market_value_2022,
            tot_mkt_val / prior_tot_mkt_val - 1 as pct_change_in_market_value_2022,
            new_construction_val / prior_tot_mkt_val as new_construction_pct_total_prior_market_value_2022,
            new_construction_val / (tot_mkt_val - prior_tot_mkt_val) as new_construction_pct_change_in_market_value_2022
        from real_acct
        where
            true
            and acct in {full_comp_tuple}
            and yr = '2022'
    ),
    cte_2023 as (
        SELECT 
            acct,
            tot_mkt_val
        from
            real_acct ra 
            inner join cte_2022 on cte_2022.account = ra.acct
        WHERE
            yr = '2023'
    )
    SELECT 
        cte_2022.*,
        cte_2023.tot_mkt_val as market_value_2023,
        cte_2023.tot_mkt_val / cte_2022.market_value_2022 - 1 as market_value_pct_change_2023
    from cte_2022
        left join cte_2023 on cte_2023.acct = cte_2022.account
'''

In [173]:
comp_df = pd.read_sql_query(comp_query, engine)

In [175]:
comp_df.sort_values(by=['market_value_pct_change_2023'])

,account,is_hcad_equity_comp,is_hcad_sale_comp,is_zwicker_comp,is_subject,street_address,market_value_2022,market_value_2021,new_construction_value_2022,change_in_market_value_2022,pct_change_in_market_value_2022,new_construction_pct_total_prior_market_value_2022,new_construction_pct_change_in_market_value_2022,market_value_2023,market_value_pct_change_2023
5,931980000216,1,1,0,0,2203 BRIARPARK DR,678852.0,621462.0,0.0,57390.0,0.092347,0.000000,0.000000,624000.0,-0.080801
9,931980000240,0,0,1,0,10035 OLYMPIA DR,552187.0,464344.0,0.0,87843.0,0.189177,0.000000,0.000000,541995.0,-0.018458
1,931930000073,1,0,0,0,10034 WICKERSHAM LN,662002.0,481146.0,148397.0,180856.0,0.375886,0.308424,0.820526,705486.0,0.065686
10,931990000274,0,1,0,0,10030 OLYMPIA DR,658092.0,546250.0,0.0,111842.0,0.204745,0.000000,0.000000,707056.0,0.074403
7,931980000235,1,0,0,0,10055 OLYMPIA DR,589338.0,580890.0,0.0,8448.0,0.014543,0.000000,0.000000,644172.0,0.093043
0,931920000058,0,0,0,1,10027 WICKERSHAM LN,652735.0,414458.0,174829.0,238277.0,0.574912,0.421826,0.733722,715818.0,0.096644
3,931940000096,0,0,1,0,10022 OVERBROOK LN,527773.0,489863.0,0.0,37910.0,0.077389,0.000000,0.000000,585726.0,0.109807
11,932000000311,0,0,1,0,10019 DEL MONTE DR,599505.0,516029.0,0.0,83476.0,0.161766,0.000000,0.000000,679428.0,0.133315
4,931940000110,0,1,1,0,10027 ELLA LEE LN,567682.0,481455.0,0.0,86227.0,0.179097,0.000000,0.000000,649335.0,0.143836
12,932020000340,0,1,0,0,10002 INWOOD DR,659540.0,514404.0,0.0,145136.0,0.282144,0.000000,0.000000,803646.0,0.218495
